___
# Classification of ED triage notes for self-harm
___

In [1]:
import pandas as pd
from scipy.stats import uniform

from sklearn.feature_selection import SelectPercentile, chi2
from sklearn.feature_extraction.text import TfidfVectorizer 

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from lightgbm import LGBMClassifier

from sklearn.pipeline import make_pipeline

import pickle

# Project imports
from self_harm_triage_notes.config import *
from self_harm_triage_notes.dev_utils import get_stopwords

# Toolbox imports
from ml_health_toolbox.dev_utils import *
from ml_health_toolbox.eval_utils import *

In [2]:
# Suppress warnings (including in worker processes started via spawn)
import os
# Ensure child processes inherit this setting
os.environ['PYTHONWARNINGS'] = 'ignore'

import warnings
warnings.filterwarnings("ignore")

In [3]:
# Set-up
# Development data
dev_data_filename = "rmh_2012_2017_dev"

# Test data
test_data_filename = "rmh_2012_2017_test"

# Additional data from LRH
lvrh_data_filename = "lvrh_2012_2022"

# Classification label and features
label = 'SH' #'SH_SI'
features = 'triage_note'
palette = ['seagreen','tomato']
    
# Undersampling
undersample = False
n_controls = 10000

# Dimensionality reduction
reduce_dim = False
lsa_method = 'svd'
n_components = 100

# Additional features
add_length = False
add_metadata = False

___
## Data preparation

In [4]:
# Load the development set
df = pd.read_parquet(proc_data_dir / (dev_data_filename + "_normalised.parquet"), engine="pyarrow")
# Keep data up to 2017 for development
df = df[df.year <= 2017].copy()

# Load addtional data from LRH and limit by year
df_lvrh = pd.read_parquet(proc_data_dir / (lvrh_data_filename + "_normalised.parquet"), engine="pyarrow")
cut_off_year = 2014
df_lvrh = df_lvrh[df_lvrh.year <= cut_off_year].copy()

# Combine datasets
df = pd.concat([df, df_lvrh], axis=0)

# Rename label column
df.rename(columns={label: 'y'}, inplace=True)

# Class names
classes = df.y.cat.categories

# Define features and labels
X = df[features]
y = df.y
groups = df.uid

# Define CV strategy
cv = get_cv_strategy(groups=groups, n_splits=N_SPLITS)

# Dictionary to store metrics
metrics_dict = {}

df.y.value_counts(normalize=True).sort_index().round(4)*100

y
Not self-harm    98.61
Self-harm         1.39
Name: proportion, dtype: float64

In [5]:
df.uid.str.startswith("RMH").value_counts()

uid
True     319288
False     82266
Name: count, dtype: int64

___
# Model development

### Quick & dirty trying a few algorithms

In [ ]:
vectorizer = TfidfVectorizer(stop_words=get_stopwords(), token_pattern=r'\S+')
selector = SelectPercentile(score_func=chi2, percentile=5)

clfs = (
    MultinomialNB(),
    LogisticRegression(solver='saga', max_iter=5000, class_weight='balanced', random_state=42),
#     RandomForestClassifier(class_weight="balanced", random_state=42),
#     GradientBoostingClassifier(random_state=42),
#     XGBClassifier(objective='binary:logistic', random_state=42),
    LGBMClassifier(force_row_wise=True, random_state=42, verbose=-1),
)

for clf in clfs:
    pipeline = make_pipeline(vectorizer, selector, clf)
    score_cv(pipeline, X, y.cat.codes, groups, len(y.cat.categories), cv=cv)

### Proper model comparison with nested CV 

In [ ]:
pipelines = []
search_modes = []
param_grids = []

vectorizer = TfidfVectorizer(stop_words='english')
selector = SelectPercentile(score_func=chi2)

# # MNB
# clf = MultinomialNB()
# pipeline = make_pipeline(vectorizer, selector, clf)
# pipelines.append(pipeline)

# search_modes.append('random')

# param_grid = {
#     'tfidfvectorizer__ngram_range': ((1, 1), (1, 2), (1, 3), (2, 2)), 
#     'tfidfvectorizer__max_df': (0.8, 0.9, 1.0),
#     'tfidfvectorizer__min_df': (10, 20),
#     'tfidfvectorizer__sublinear_tf': (True, False),
#     'selectpercentile__percentile': (0.1, 0.5, 1, 5),
#     'multinomialnb__alpha': (1e-5, 1e-2, 1e-1, 0.25, 0.5, 1, 10, 50, 100),
#     }
# param_grids.append(param_grid)

# # Logistic Regression
# clf = LogisticRegression(solver='saga', max_iter=5000, class_weight='balanced', random_state=42)
# pipeline = make_pipeline(vectorizer, selector,clf)
# pipelines.append(pipeline)

# search_modes.append('random')

# param_grid = {
#     'tfidfvectorizer__ngram_range': ((1, 1), (1, 2), (1, 3), (2, 2)), 
#     'tfidfvectorizer__max_df': (0.8, 0.9, 1.0),
#     'tfidfvectorizer__min_df': (10, 20),
#     'tfidfvectorizer__sublinear_tf': (True, False),
#     'selectpercentile__percentile': (0.1, 0.5, 1, 5),
#     'logisticregression__C': (1e-5, 1e-2, 1e-1, 0.25, 0.5, 1, 10, 50, 100),
#     'logisticregression__l1_ratio': (1.0, 0.75, 0.5, 0.25, 0.0),
#     }
# param_grids.append(param_grid)

# LightGBM
clf = LGBMClassifier(force_row_wise=True, random_state=42, verbose=-1)
pipeline = make_pipeline(vectorizer, selector, clf)
pipelines.append(pipeline)

search_modes.append('random')

num_leaves = [5, 10, 30, 50, 100]
max_depth = [-1, 3, 4, 5, 6, 7, 10]
learning_rate = [0.01, 0.1, 0.25]
n_estimators = [300, 350, 400, 450, 500]
min_child_samples = range(1, 50)
subsample = uniform(0.1, 1.0)
colsample_bytree = uniform(0.1, 1.0) 
reg_alpha = uniform(0.0, 1.0)
reg_lambda = uniform(0.0, 1.0)

param_grid = {
    'tfidfvectorizer__ngram_range': ((1, 1), (1, 2), (1, 3), (2, 2)), 
    'tfidfvectorizer__max_df': (0.8, 0.9, 1.0),
    'tfidfvectorizer__min_df': (10, 20),
    'tfidfvectorizer__sublinear_tf': (True, False),
    'selectpercentile__percentile': (0.1, 0.5, 1, 5),
    'lgbmclassifier__num_leaves': num_leaves,
    'lgbmclassifier__max_depth': max_depth,
    'lgbmclassifier__learning_rate': learning_rate,
    'lgbmclassifier__n_estimators': n_estimators,
    'lgbmclassifier__min_child_samples': min_child_samples,
    'lgbmclassifier__subsample': subsample,
    'lgbmclassifier__colsample_bytree': colsample_bytree,
    'lgbmclassifier__reg_alpha': reg_alpha,
    'lgbmclassifier__reg_lambda': reg_lambda,
}
param_grids.append(param_grid)

In [ ]:
for pipeline, search_mode, param_grid in zip(pipelines, search_modes, param_grids):
        benchmark_nested_cv(pipeline, search_mode, param_grid, X, y.cat.codes, groups, len(y.cat.categories), cv=cv)

### Tune the final model

In [6]:
# LightGBM
vectorizer = TfidfVectorizer(stop_words=get_stopwords(), token_pattern=r'\S+')
selector = SelectPercentile(score_func=chi2)
clf = LGBMClassifier(force_row_wise=True, random_state=42, verbose=-1)
pipeline = make_pipeline(vectorizer, selector, clf)

num_leaves = [5, 10, 30, 50, 100]
max_depth = [-1, 3, 4, 5, 6, 7, 10]
learning_rate = [0.1]
n_estimators = [300, 400, 500]
min_child_samples = range(1, 50)
subsample = uniform(0.5, 1.0)
colsample_bytree = uniform(0.5, 1.0) 
reg_alpha = uniform(0.0, 1.0)
reg_lambda = uniform(0.0, 1.0)

param_grid = {
    'tfidfvectorizer__ngram_range': ((1, 1), (1, 2), (1, 3), (2, 2)), 
    'tfidfvectorizer__max_df': (0.8, 0.9, 1.0),
    'tfidfvectorizer__min_df': (10, 20),
    'tfidfvectorizer__sublinear_tf': (True, False),
    'selectpercentile__percentile': uniform(0.1, 15.0),
    'lgbmclassifier__num_leaves': num_leaves,
    'lgbmclassifier__max_depth': max_depth,
    'lgbmclassifier__learning_rate': learning_rate,
    'lgbmclassifier__n_estimators': n_estimators,
    'lgbmclassifier__min_child_samples': min_child_samples,
    'lgbmclassifier__subsample': subsample,
    'lgbmclassifier__colsample_bytree': colsample_bytree,
    'lgbmclassifier__reg_alpha': reg_alpha,
    'lgbmclassifier__reg_lambda': reg_lambda,
}

search_result = search_params(pipeline, 'random', param_grid, X, y.cat.codes, groups, cv=cv, scoring='average_precision')
tuned_model = search_result.best_estimator_

[LightGBM] [Fatal] Check failed: (bagging_fraction) <= (1.0) at /Users/runner/miniforge3/conda-bld/bld/rattler-build_liblightgbm_1768022048/work/src/io/config_auto.cpp, line 367 .

[LightGBM] [Fatal] Check failed: (bagging_fraction) <= (1.0) at /Users/runner/miniforge3/conda-bld/bld/rattler-build_liblightgbm_1768022048/work/src/io/config_auto.cpp, line 367 .

[LightGBM] [Fatal] Check failed: (bagging_fraction) <= (1.0) at /Users/runner/miniforge3/conda-bld/bld/rattler-build_liblightgbm_1768022048/work/src/io/config_auto.cpp, line 367 .

[LightGBM] [Fatal] Check failed: (bagging_fraction) <= (1.0) at /Users/runner/miniforge3/conda-bld/bld/rattler-build_liblightgbm_1768022048/work/src/io/config_auto.cpp, line 367 .

[LightGBM] [Fatal] Check failed: (bagging_fraction) <= (1.0) at /Users/runner/miniforge3/conda-bld/bld/rattler-build_liblightgbm_1768022048/work/src/io/config_auto.cpp, line 367 .

[LightGBM] [Fatal] Check failed: (bagging_fraction) <= (1.0) at /Users/runner/miniforge3/conda-

Best for current fold: 0.812 using {'lgbmclassifier__colsample_bytree': 0.6437533917594654, 'lgbmclassifier__learning_rate': 0.1, 'lgbmclassifier__max_depth': -1, 'lgbmclassifier__min_child_samples': 18, 'lgbmclassifier__n_estimators': 400, 'lgbmclassifier__num_leaves': 100, 'lgbmclassifier__reg_alpha': 0.9078562904449273, 'lgbmclassifier__reg_lambda': 0.6037369092256085, 'lgbmclassifier__subsample': 0.874138899721232, 'selectpercentile__percentile': 11.012408892142915, 'tfidfvectorizer__max_df': 0.9, 'tfidfvectorizer__min_df': 10, 'tfidfvectorizer__ngram_range': (1, 2), 'tfidfvectorizer__sublinear_tf': False}
nan (+/- nan) with: {'lgbmclassifier__colsample_bytree': 1.0717945146506969, 'lgbmclassifier__learning_rate': 0.1, 'lgbmclassifier__max_depth': 6, 'lgbmclassifier__min_child_samples': 30, 'lgbmclassifier__n_estimators': 400, 'lgbmclassifier__num_leaves': 10, 'lgbmclassifier__reg_alpha': 0.01289752471949479, 'lgbmclassifier__reg_lambda': 0.8718088046500656, 'lgbmclassifier__subsam

>**Optimising on all RMH dev + 2012 LVRH**
>
>Best for current fold: 0.798 using {'lgbmclassifier__colsample_bytree': 0.974039388225595, 'lgbmclassifier__learning_rate': 0.1, 'lgbmclassifier__max_depth': 5, 'lgbmclassifier__min_child_samples': 1, 'lgbmclassifier__n_estimators': 500, 'lgbmclassifier__num_leaves': 10, 'lgbmclassifier__reg_alpha': 0.42367924416491276, 'lgbmclassifier__reg_lambda': 0.959474192232794, 'lgbmclassifier__subsample': 0.7051575051445674, 'selectpercentile__percentile': 15.026537562825311, 'tfidfvectorizer__max_df': 0.9, 'tfidfvectorizer__min_df': 10, 'tfidfvectorizer__ngram_range': (1, 1), 'tfidfvectorizer__sublinear_tf': True}
>
>**Optimising on all RMH dev + 2012-2014 LVRH**
>
>Best for current fold: 0.812 using {'lgbmclassifier__colsample_bytree': 0.6437533917594654, 'lgbmclassifier__learning_rate': 0.1, 'lgbmclassifier__max_depth': -1, 'lgbmclassifier__min_child_samples': 18, 'lgbmclassifier__n_estimators': 400, 'lgbmclassifier__num_leaves': 100, 'lgbmclassifier__reg_alpha': 0.9078562904449273, 'lgbmclassifier__reg_lambda': 0.6037369092256085, 'lgbmclassifier__subsample': 0.874138899721232, 'selectpercentile__percentile': 11.012408892142915, 'tfidfvectorizer__max_df': 0.9, 'tfidfvectorizer__min_df': 10, 'tfidfvectorizer__ngram_range': (1, 2), 'tfidfvectorizer__sublinear_tf': False}
>
>**Optimising on all RMH dev + 2012-2015 LVRH**
>
>Best for current fold: 0.802 using {'lgbmclassifier__colsample_bytree': 0.7813777132071821, 'lgbmclassifier__learning_rate': 0.1, 'lgbmclassifier__max_depth': 10, 'lgbmclassifier__min_child_samples': 27, 'lgbmclassifier__n_estimators': 500, 'lgbmclassifier__num_leaves': 30, 'lgbmclassifier__reg_alpha': 0.7255883170393342, 'lgbmclassifier__reg_lambda': 0.36850255502459706, 'lgbmclassifier__subsample': 0.5945175754829907, 'selectpercentile__percentile': 4.3052091262543195, 'tfidfvectorizer__max_df': 1.0, 'tfidfvectorizer__min_df': 20, 'tfidfvectorizer__ngram_range': (1, 1), 'tfidfvectorizer__sublinear_tf': True}
>
>**Optimising on all RMH dev + 2012-2017 LVRH**
>
>Best for current fold: 0.814 using {'lgbmclassifier__colsample_bytree': 0.608609874153036, 'lgbmclassifier__learning_rate': 0.1, 'lgbmclassifier__max_depth': 10, 'lgbmclassifier__min_child_samples': 37, 'lgbmclassifier__n_estimators': 500, 'lgbmclassifier__num_leaves': 10, 'lgbmclassifier__reg_alpha': 0.6002691532950243, 'lgbmclassifier__reg_lambda': 0.8517816444662138, 'lgbmclassifier__subsample': 0.9621660963337506, 'selectpercentile__percentile': 9.870609668796567, 'tfidfvectorizer__max_df': 1.0, 'tfidfvectorizer__min_df': 20, 'tfidfvectorizer__ngram_range': (1, 3), 'tfidfvectorizer__sublinear_tf': False}


### Tuned model

In [ ]:
vectorizer = TfidfVectorizer(stop_words=get_stopwords(), token_pattern=r'\S+')
selector = SelectPercentile(score_func=chi2)
clf = LGBMClassifier(force_row_wise=True, random_state=42, verbose=-1)
pipeline = make_pipeline(vectorizer, selector, clf)
params = {'lgbmclassifier__colsample_bytree': 0.974039388225595, 'lgbmclassifier__learning_rate': 0.1, 'lgbmclassifier__max_depth': 5, 'lgbmclassifier__min_child_samples': 1, 'lgbmclassifier__n_estimators': 500, 'lgbmclassifier__num_leaves': 10, 'lgbmclassifier__reg_alpha': 0.42367924416491276, 'lgbmclassifier__reg_lambda': 0.959474192232794, 'lgbmclassifier__subsample': 0.7051575051445674, 'selectpercentile__percentile': 15.026537562825311, 'tfidfvectorizer__max_df': 0.9, 'tfidfvectorizer__min_df': 10, 'tfidfvectorizer__ngram_range': (1, 1), 'tfidfvectorizer__sublinear_tf': True}

tuned_model = pipeline.set_params(**params)

 ### Evaluate the tuned model

In [7]:
# Calculate performance scores
score_cv(tuned_model, X, y.cat.codes, groups, len(y.cat.categories), cv=cv)

# Make predictions for each CV fold
y_proba = predict_cv(tuned_model, X, y.cat.codes, groups, cv=cv)

________________________________________________________________________________
Training with 5-fold cross-validation:
Pipeline(steps=[('tfidfvectorizer',
                 TfidfVectorizer(max_df=0.9, min_df=10, ngram_range=(1, 2),
                                 stop_words=['a', 'about', 'above', 'across',
                                             'after', 'afterwards', 'again',
                                             'against', 'ain', 'all', 'almost',
                                             'alone', 'along', 'already',
                                             'also', 'although', 'always', 'am',
                                             'among', 'amongst', 'amoungst',
                                             'amount', 'an', 'and', 'another',
                                             'any', 'anyhow', 'anyone',
                                             'anything', 'anyway...
                ('selectpercentile',
                 SelectPercentile(percentile=

### Calibrate the tuned model

In [ ]:
# Calibrate the model
tuned_model_calibrated, y_proba_calibrated = calibrate_cv(tuned_model, X, y, groups, cv=cv, method='isotonic', ensemble=True)

In [ ]:
# Calculate calibration curves for the uncalibrated and calibrated versions of the model
calibration_curves = prepare_calibration_curve(y, y_proba, y_proba_calibrated)
# Plot calibration curves to compare the uncalibrate and calibrated vesions of the model
plot_calibration_curve(calibration_curves, palette=['#C5D1E0', '#757F8D'], results_dir=results_dir, filename=dev_data_filename + "comparison")

### Select the best model

In [8]:
# Use the uncalibrated or calibrated version
best_model = tuned_model
y_proba = y_proba

### Re-train the final model on the full development set

In [9]:
# Re-train the final model on the full development set
best_model.fit(X, y.cat.codes)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('tfidfvectorizer', ...), ('selectpercentile', ...), ...]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"input input: {'filename', 'file', 'content'}, default='content'- If `'filename'`, the sequence passed as an argument to fit is expected to be a list of filenames that need reading to fetch the raw content to analyze.- If `'file'`, the sequence items must have a 'read' method (file-like object) that is called to fetch the bytes in memory.- If `'content'`, the input is expected to be a sequence of items that can be of type string or byte.",'content'
,"encoding encoding: str, default='utf-8'If bytes or files are given to analyze, this encoding is used todecode.",'utf-8'
,"decode_error decode_error: {'strict', 'ignore', 'replace'}, default='strict'Instruction on what to do if a byte sequence is given to analyze thatcontains characters not of the given `encoding`. By default, it is'strict', meaning that a UnicodeDecodeError will be raised. Othervalues are 'ignore' and 'replace'.",'strict'
,"strip_accents strip_accents: {'ascii', 'unicode'} or callable, default=NoneRemove accents and perform other character normalizationduring the preprocessing step.'ascii' is a fast method that only works on characters that havea direct ASCII mapping.'unicode' is a slightly slower method that works on any characters.None (default) means no character normalization is performed.Both 'ascii' and 'unicode' use NFKD normalization from:func:`unicodedata.normalize`.",None
,"lowercase lowercase: bool, default=TrueConvert all characters to lowercase before tokenizing.",True
,"preprocessor preprocessor: callable, default=NoneOverride the preprocessing (string transformation) stage whilepreserving the tokenizing and n-grams generation steps.Only applies if ``analyzer`` is not callable.",None
,"tokenizer tokenizer: callable, default=NoneOverride the string tokenization step while preserving thepreprocessing and n-grams generation steps.Only applies if ``analyzer == 'word'``.",None


___
# Model evaluation

### RMH test set

In [10]:
# Load the test set
df_test = pd.read_parquet(proc_data_dir / (test_data_filename + "_normalised.parquet"), engine="pyarrow")

# Keep data up to 2017 for evaluation
df_test = df_test[df_test.year <= 2017].copy()

# Rename label column
df_test.rename(columns={label: 'y'}, inplace=True)

# Make predictions
df_test[classes], df_test['prediction'] = make_predictions(best_model, df_test[features], classes, 0.3, normalize=True)

# Define curves to calculate
curves_to_calculate = [
        {
        'name': 'PR AUC', 
        'func': calculate_pr_curve_coords,
        'binary_filter': True,
        },
    ]

# Calculate curve coordinates for the full dataset w/bootstrap
curve_coords = calculate_curve_coords(df_test.y, df_test[classes], curves_to_calculate, generator=True)

# Calculate discrimination metrics per class
calculate_stats(calculate_auc_from_coords(curve_coords), return_ci=True).round(2)

,class_,metric,mean,lower_bound,upper_bound
0,Self-harm,PR AUC,0.8,0.78,0.82


### Remaining LRH data

In [11]:
# Load the test set
df_test = pd.read_parquet(proc_data_dir / (lvrh_data_filename + "_normalised.parquet"), engine="pyarrow")

# Keep data up to 2017 for evaluation
df_test = df_test[df_test.year > cut_off_year].reset_index(drop=True).copy()

# Rename label column
df_test.rename(columns={label: 'y'}, inplace=True)

# Make predictions
df_test[classes], df_test['prediction'] = make_predictions(best_model, df_test[features], classes, 0.3, normalize=True)

# Define curves to calculate
curves_to_calculate = [
        {
        'name': 'PR AUC', 
        'func': calculate_pr_curve_coords,
        'binary_filter': True,
        },
    ]

# Calculate curve coordinates for the full dataset w/bootstrap
curve_coords = calculate_curve_coords(df_test.y, df_test[classes], curves_to_calculate, generator=True)

# Calculate discrimination metrics per class
calculate_stats(calculate_auc_from_coords(curve_coords), return_ci=True).round(2)

,class_,metric,mean,lower_bound,upper_bound
0,Self-harm,PR AUC,0.82,0.8,0.83
